In [14]:
# Importando bibliotecas
import dask
import warnings
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import dask.dataframe as dd

from rapidfuzz import process

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option("display.max_colwidth", None)

# Verifica o uso de CPU
dask.system.cpu_count()

10

# Dados

Os dados utilizados na pesquisa e resultado de um cruzamento de dados, de origens publica e restrita tendo como fontes o portal do [**Justiça e Números - Indicadores**](https://justica-em-numeros.cnj.jus.br/painel-estatisticas/), [**Portal da Tranparência do TJBA**](https://www.tjba.jus.br/transparencia/), [**DataJude**}(https://www.cnj.jus.br/sistemas/datajud/) e PJe em parceria com o TJBA.

Os dados no Justiça e Números foram extraidos em 11/08/2025 com informações do: Painel atualizado em 25/07/2025, com dados recebidos dos tribunais e processados até 16/07/2025 (situações processuais até 30/06/2025)

Diretódio do projeto onde estão os dados:

```
dados
├── MEMBROS-DO-TJBA-E-AGENTES-PUBLICOS.csv - Arquivo do PT com lista de magistrados e lotação
└── CNJ_TJBA_JULGADOS.csv - Arquivo do JN com processos julgados TJBA já filtrados do 1º/TJBA
```

## Dados de Magistrados - Portal da Transparência
Esses dados podem demosntrar quantos magistrados exitem por vara.

In [2]:
# Inportando dados de magistrados do PT
df_magistrados = pd.read_csv('../../dados/MEMBROS_E_AGENTES_PUBLICOS_JUL2025.csv', sep=';')
df_magistrados = df_magistrados[df_magistrados['Cargo'].str.startswith('JUIZ', na=False)]

df_magistrados.head()

,Nome,Matrícula Funcional,Cargo,Função de Confiança/ Cargo em Comissão,Lotação,Ato de Provimento,Data de Publicação
7,ABRAAO BARRETO CORDEIRO,9679782,JUIZ DE DIREITO,NaN,VARA CRIM JURI EXEC PENAIS MEN,PORT S/N,16/09/2013
25,ADALBERTO LIMA BORGES FILHO,9698345,JUIZ DE DIREITO,NaN,JURISDICAO PLENA,PORT S/N,06/10/2021
69,ADERALDO DE MORAIS LEITE JUNIOR,9679871,JUIZ DE DIREITO,NaN,JURISDICAO PLENA,PORT S/N,16/09/2013
73,ADIANE JAQUELINE NEVES DA SILVA OLIVEIRA,9679812,JUIZ DE DIREITO,NaN,1ª VARA DOS SISTEMA DOS JUIZADOS ESPECIAIS,PORT S/N,16/09/2013
74,ADIDA ALVES DOS SANTOS,8098654,JUIZ DE DIREITO,NaN,2ª VARA DA INFANCIA E JUVENTUDE,PORT S/N,01/12/2004


In [18]:
df_magistrados.shape

(621, 7)

In [17]:
df_magistrados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 621 entries, 7 to 8846
Data columns (total 7 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Nome                                    621 non-null    object
 1   Matrícula Funcional                     621 non-null    int64 
 2   Cargo                                   621 non-null    object
 3   Função de Confiança/ Cargo em Comissão  0 non-null      object
 4   Lotação                                 621 non-null    object
 5   Ato de Provimento                       621 non-null    object
 6   Data de Publicação                      621 non-null    object
dtypes: int64(1), object(6)
memory usage: 38.8+ KB


In [3]:
# Contagens de Juízes
df_magistrados.Cargo.value_counts()

Cargo
JUIZ DE DIREITO                                      544
JUIZ SUBSTITUTO                                       56
JUIZ SUBSTITUTO DE SEGUNDO GRAU - ENTRANCIA FINAL     21
Name: count, dtype: int64

In [4]:
# Pesquisa por magistrado
df_magistrados[df_magistrados['Nome'] == 'ULYSSES MAYNARD SALGADO']

,Nome,Matrícula Funcional,Cargo,Função de Confiança/ Cargo em Comissão,Lotação,Ato de Provimento,Data de Publicação
8430,ULYSSES MAYNARD SALGADO,8088900,JUIZ DE DIREITO,NaN,1A VARA FAZENDA PUBLICA,PORT S/N,23/12/2002


In [5]:
# Contagem de Lotação
df_magistrados.groupby('Lotação').agg(quant=('Lotação', 'count'))

,quant
Lotação,
1ª VARA CRIM JURI EXEC PENAIS,1
VARA EXEC. PENAIS E MEDIDAS ALTERNATIVAS,1
10ª VARA CIVEL,1
10ª VARA DE FAMILIA,1
10ª VARA DE RELACOES DE CONSUMO,1
...,...
VARA ESP CRIM INFAN JUVEN,1
VARA FEITOS RELATIVOS REL DE CONSUMO CIVEIS E COM,1
VARA FEITOS RELATIVOS REL DE CONSUMO CIVEIS E COM,1


## Dados de Processos Julgados - Indicadores CNJ

In [ ]:
# Importando dados do CNJ, processos julgados
df_cnj = pd.read_csv('../../dados/CNJ_TJBA_JULGADOS.csv', encoding='utf8', sep=';')
df_cnj.head()

,Tribunal,Grau,Nome_Orgao,UF,Municipio,Ano,Mes,Processo,Codigos_classes,Codigo_da_Ultima_classe,Nome_da_Ultima_classe,Codigos_assuntos,Data_de_Referencia,Data_do_julgamento:_Situacao:_Movimento,Data_da_decisao:_Situacao:_Movimento,Formato,Procedimento,Recurso,Codigo_Orgao,id_municipio,Polo_ativo,Polo_ativo_-_CNPJ,Polo_ativo_-_Natureza_juridica,Polo_ativo_-_CNAE,Polo_passivo,Polo_passivo_-_CNPJ,Polo_passivo_-_Natureza_juridica,Polo_passivo_-_CNAE,Poder_publico
0,TJBA,G1,VARA DE JURISDIÇÃO PLENA - ITAPICURU,BA,ITAPICURU,2024,3,8000013-59.2017.8.05.0127,{1116},1116,EXECUÇÃO FISCAL,{5952},2024-03-26,{20240326:28:459},{},Eletrônico,Execução fiscal,0,5312,2349,{MUNICIPIO DE ITAPICURU},{13647557000160},{MUNICIPIO},{ADMINISTRACAO PUBLICA EM GERAL},NaN,NaN,NaN,NaN,Poder_Publico-Polo_Ativo
1,TJBA,G1,VARA DE JURISDIÇÃO PLENA - ITAPICURU,BA,ITAPICURU,2024,4,8000014-44.2017.8.05.0127,{1116},1116,EXECUÇÃO FISCAL,{5952},2024-04-10,{20240410:28:461},{},Eletrônico,Execução fiscal,0,5312,2349,{MUNICIPIO DE ITAPICURU},{13647557000160},{MUNICIPIO},{ADMINISTRACAO PUBLICA EM GERAL},NaN,NaN,NaN,NaN,Poder_Publico-Polo_Ativo
2,TJBA,G1,VARA DE JURISDIÇÃO PLENA - ITAPICURU,BA,ITAPICURU,2024,3,8000894-36.2017.8.05.0127,{1116},1116,EXECUÇÃO FISCAL,{5952},2024-03-26,{20240326:28:459},{},Eletrônico,Execução fiscal,0,5312,2349,{MUNICIPIO DE ITAPICURU},{13647557000160},{MUNICIPIO},{ADMINISTRACAO PUBLICA EM GERAL},NaN,NaN,NaN,NaN,Poder_Publico-Polo_Ativo
3,TJBA,G1,VARA DE JURISDIÇÃO PLENA - ITAPICURU,BA,ITAPICURU,2024,3,8000012-74.2017.8.05.0127,{1116},1116,EXECUÇÃO FISCAL,{5952},2024-03-26,{20240326:28:459},{},Eletrônico,Execução fiscal,0,5312,2349,{MUNICIPIO DE ITAPICURU},{13647557000160},{MUNICIPIO},{ADMINISTRACAO PUBLICA EM GERAL},NaN,NaN,NaN,NaN,Poder_Publico-Polo_Ativo
4,TJBA,G1,VARA DE JURISDIÇÃO PLENA - ITAPICURU,BA,ITAPICURU,2024,3,0000753-32.2012.8.05.0127,{1116},1116,EXECUÇÃO FISCAL,{10887},2024-03-26,{20240326:28:459},{},Eletrônico,Execução fiscal,0,5312,2349,{INSTITUTO NACIONAL DE METROLOGIA QUALIDADE E TECNOLOGIA - INMETRO.},{00662270000168},{AUTARQUIA FEDERAL},{ADMINISTRACAO PUBLICA EM GERAL},{G D DE SOUZA DE ITAPICURU},{07593292000162},{EMPRESARIO (INDIVIDUAL)},{FABRICACAO DE PRODUTOS DE PADARIA E CONFEITARIA COM PREDOMINANCIA DE PRODUCAO PROPRIA},Poder_Publico-Polo_Ativo


In [ ]:
df_cnj.shape

(1440265, 29)

In [ ]:
# Processos que tiveream mais de uma decisão
df_cnj['Processo'].value_counts()[lambda x: x > 1]

Processo
0700281-17.2021.8.05.0113    12
8005064-54.2021.8.05.0113     7
0001917-26.2002.8.05.0113     7
0011275-05.2008.8.05.0113     7
8005315-83.2022.8.05.0001     6
0018977-65.2009.8.05.0113     6
8006637-59.2023.8.05.0113     6
0502184-42.2019.8.05.0113     5
0302815-43.2014.8.05.0113     5
0001540-63.2010.8.05.0052     5
0500831-98.2018.8.05.0113     5
0002537-33.2005.8.05.0113     5
8009646-29.2023.8.05.0113     5
0005678-50.2011.8.05.0113     5
0500164-83.2016.8.05.0113     5
8000342-96.2024.8.05.0201     5
8000383-09.2024.8.05.0219     4
8008937-89.2023.8.05.0146     4
8000563-84.2021.8.05.0007     4
0003811-85.2012.8.05.0113     4
sigiloso(602979994)           4
8001076-07.2021.8.05.0022     4
0301982-86.2015.8.05.0146     4
0559193-80.2014.8.05.0001     4
8014796-07.2021.8.05.0001     4
8000358-05.2024.8.05.0119     4
0539240-33.2014.8.05.0001     4
0308570-82.2013.8.05.0113     4
8068691-09.2023.8.05.0001     4
0014563-92.2007.8.05.0113     4
0006607-20.2010.8.05.0113     4

## Cruzamento dos Dados

Nessa fase precisamos cruzar alguns dados, apesar de termos fontes como CNJ e TJBA, algumas informações são mais específicas. Nessa primeira etapa devido ao volume e acomplexidade, foi utilizado a ferramenta de ETL [**Apache Hop**](https://hop.apache.org/), para o cruzamento, análises e um tratamento inicial. O artefado final gerado nessa etapa será um arquivo parquet, que usaremos como principal fonte de dados para o modelo.

<p align="center">
    <img width="850" height="250" src="../../img/001.png">
</p>

## Tratamento dos Dados

Aqui vamos trabalhar com os dados realizando alguns tratamentos, e adequando os dados;

*	Remover colunas (features) dos df's.
*	No aquivo baixado da Portal da Tranarência temos uma matrícula e nome que identifica os magistrados, porem essa matrícula não aparece nos outros dados, são códigos diferentes. 
*	Levar o nomes das varas do df pricipal para o df dos magistrados, pois tem erros de gigitação qua vão atrapalhar.
*	Levar a identificação da comarca para o df dos magistrados.
*	Tratamento nas colunas, verificar o que pode nos atrapalhar na utilização dos dados.






### Tratando dados do TJBA

In [2]:
# Verificando eschema do arquivo parquet
table = pq.read_table("../../dados/DADOS_TJBA_PG.parquet")
print(table.schema)

pje_processo: string
vm_processo: string
vm_dt_recebimento: timestamp[ms, tz=UTC]
vm_dt_movimento: timestamp[ms, tz=UTC]
vm_cod_cnj_comarca: int64
vm_mapa_comarca: int64
vm_cod_comarca: int64
vm_comarca: string
pje_id_jurisdicao: int64
pje_ds_jurisdicao: string
vm_mapa_vara: int64
vm_cod_vara: int64
vm_cod_cnj_vara: int64
pje_id_orgao_julgador: int64
pje_nr_vara: int64
vm_vara: string
pje_ds_orgao_julgador: string
pje_cd_classe_judicial: string
vm_cod_cls_cnj: int64
vm_classe: string
pje_ds_classe_judicial: string
pje_ds_natureza: string
pje_vl_peso: int64
pje_cd_assunto_principal: string
pje_ds_assunto_principal: string
vm_cod_mov_cnj: int64
vm_movimento: string
pje_id_competencia: int64
pje_ds_competencia: string
vm_competencia: string
vm_localizacao: string
vm_tp_proc: string
pje_id_orgao_julgador_cargo: int64
pje_ds_orgao_julgador_cargo: string
vm_cod_cnj_magistrado: int64
vm_mapa_magistrado: int64
vm_cod_magistrado: string
vm_magistrado: string
vm_consulta: string
vm_situacao: str

In [2]:
# Importando arquivo parquet gerado pelo Apache Hop Pandas
df_tjba = pd.read_parquet('../../dados/DADOS_TJBA_PG.parquet', engine="pyarrow", use_pandas_metadata=False)

# Importando arquivo parquet gerado pelo Apache Hop Dask
#df_tjba = dd.read_parquet('../../dados/DADOS_TJBA_PG.parquet', engine="pyarrow", use_pandas_metadata=False)

# Conver timestamp para datetime
#df["cnj_dt_referencia"] = df["cnj_dt_referencia"].dt.tz_convert(None)
#df["vm_dt_recebimento"] = df["vm_dt_recebimento"].dt.tz_convert(None)
#df["vm_dt_movimento"] = df["vm_dt_movimento"].dt.tz_convert(None)

In [3]:
# Removendo fetures Pandas
df_tjba.drop([
    'pje_processo', 'vm_cod_cnj_comarca', 'vm_mapa_comarca', 'pje_id_jurisdicao', 'pje_ds_jurisdicao', 
    'vm_mapa_vara', 'vm_cod_cnj_vara', 'pje_id_orgao_julgador', 'pje_nr_vara', 'pje_ds_orgao_julgador', 
    'pje_cd_classe_judicial', 'pje_ds_classe_judicial', 'pje_vl_peso', 'pje_id_competencia', 
    'pje_ds_competencia', 'vm_cod_cnj_magistrado', 'vm_cod_magistrado', 'vm_entrancia', 'vm_compl_mov'
], axis=1, inplace=True)

# Removendo fetures Dask
#df_tjba = df_tjba.drop([
#    'pje_processo', 'vm_cod_cnj_comarca', 'vm_mapa_comarca', 'pje_id_jurisdicao', 'pje_ds_jurisdicao', 
#    'vm_mapa_vara', 'vm_cod_cnj_vara', 'pje_id_orgao_julgador', 'pje_nr_vara', 'pje_ds_orgao_julgador', 
#    'pje_cd_classe_judicial', 'pje_ds_classe_judicial', 'pje_vl_peso', 'pje_id_competencia', 
#    'pje_ds_competencia', 'vm_cod_cnj_magistrado', 'vm_cod_magistrado', 'vm_entrancia', 'vm_compl_mov'
#], axis=1)

In [4]:
df_tjba.head()

,vm_processo,vm_dt_recebimento,vm_dt_movimento,vm_cod_comarca,vm_comarca,vm_cod_vara,vm_vara,vm_cod_cls_cnj,vm_classe,pje_ds_natureza,pje_cd_assunto_principal,pje_ds_assunto_principal,vm_cod_mov_cnj,vm_movimento,vm_competencia,vm_localizacao,vm_tp_proc,pje_id_orgao_julgador_cargo,pje_ds_orgao_julgador_cargo,vm_mapa_magistrado,vm_magistrado,vm_consulta,vm_situacao,vm_tempo_tramitacao,pje_vl_peso_processual,vm_prioridade,vm_ds_prioridade,pje_vl_peso_prioridade,vm_concluso,vm_sistema,vm_num_ativo
0,8003619-75.2022.8.05.0271,2022-09-19 14:02:01+00:00,2024-08-26 12:47:34+00:00,1255,VALENCA,11534,2ª VARA DE FEITOS DE REL DE CONS. CÍVEL E COMERCIAIS,65,ACAO CIVIL PUBLICA,CONHECIMENTO,10113,Flora,11010,PROFERIDO DESPACHO DE MERO EXPEDIENTE,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,DIREÇÃO DE SECRETARIA,DIGITAL,1125.0,Juiz de Direito Titular,378.0,LEONARDO RULIAN CUSTODIO,DESPACHO,EM ANDAMENTO,0.0,4.0,N,None,0.0,0,PJEPG,1
1,8003068-15.2022.8.05.0039,2022-02-06 18:58:24+00:00,2024-09-19 15:03:05+00:00,115,CAMACARI,11453,2ª VARA DA FAZENDA PÚBLICA,1116,EXECUCAO FISCAL,EXECUçãO,6017,Dívida Ativa (Execução Fiscal),461,EXTINTO O PROCESSO POR AUSENCIA DAS CONDICOES DA ACAO,FAZENDA PUBLICA TRIBUTOS,DIREÇÃO DE SECRETARIA,DIGITAL,935.0,JUIZ DE DIREITO,166.0,DANIEL LIMA FALCAO,JULGAMENTO,BAIXADO,0.0,4.0,N,None,0.0,0,PJEPG,1
2,0088391-11.2003.8.05.0001,2003-07-18 13:10:09+00:00,2025-07-18 22:24:10+00:00,18,SALVADOR,111452,15ª VARA DA FAZENDA PÚBLICA,1116,EXECUCAO FISCAL,EXECUçãO,12989,Execução Fiscal,11010,PROFERIDO DESPACHO DE MERO EXPEDIENTE,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,DIREÇÃO DE SECRETARIA,DIGITAL,2853.0,Juiz de Direito Titular,161.0,CRISTIANE MENEZES SANTOS BARRETO,DESPACHO,EM ANDAMENTO,0.0,4.0,N,None,0.0,0,PJEPG,1
3,0000087-73.2019.8.05.0066,2019-03-18 18:20:00+00:00,2025-07-09 17:53:34+00:00,1165,CONDEUBA,11754,VARA JURISDIÇÃO PLENA,283,ACAO PENAL PROCEDIMENTO ORDINARIO,CONHECIMENTO,11417,Estupro de vulnerável,11010,PROFERIDO DESPACHO DE MERO EXPEDIENTE,CRIME,DIREÇÃO DE SECRETARIA,DIGITAL,1740.0,JUIZ SUBSTITUTO,1492.0,CARLOS TIAGO SILVA ADAES,DESPACHO,EM ANDAMENTO,0.0,4.0,N,None,0.0,0,PJEPG,1
4,8055437-37.2021.8.05.0001,2021-05-28 13:47:59+00:00,2025-05-30 17:27:43+00:00,18,SALVADOR,11553,6ª VARA CÍVEL E COMERCIAL,7,PROCEDIMENTO COMUM CIVEL,CONHECIMENTO,9597,Seguro,11010,PROFERIDO DESPACHO DE MERO EXPEDIENTE,CIVEL,DIREÇÃO DE SECRETARIA,DIGITAL,1165.0,Juiz de Direito Titular,750.0,CARLOS CARVALHO RAMOS DE CERQUEIRA JUNIOR,DESPACHO,EM ANDAMENTO,0.0,4.0,N,None,0.0,0,PJEPG,1


In [5]:
df_tjba.shape

(11715246, 31)

In [6]:
df_tjba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11715246 entries, 0 to 11715245
Data columns (total 31 columns):
 #   Column                       Dtype              
---  ------                       -----              
 0   vm_processo                  object             
 1   vm_dt_recebimento            datetime64[ms, UTC]
 2   vm_dt_movimento              datetime64[ms, UTC]
 3   vm_cod_comarca               int64              
 4   vm_comarca                   object             
 5   vm_cod_vara                  int64              
 6   vm_vara                      object             
 7   vm_cod_cls_cnj               int64              
 8   vm_classe                    object             
 9   pje_ds_natureza              object             
 10  pje_cd_assunto_principal     object             
 11  pje_ds_assunto_principal     object             
 12  vm_cod_mov_cnj               int64              
 13  vm_movimento                 object             
 14  vm_competencia  

In [7]:
# Algumas colunas tem valores None indicando ausência de dados, mudando para NaN
#df_tjba['vm_magistrado'] = df_tjba['vm_magistrado'].apply(lambda x: np.nan if x is None else x)

# Colocando dados em letras maiúsculas
df_tjba['pje_ds_assunto_principal'] = df_tjba['pje_ds_assunto_principal'].str.upper()
df_tjba['pje_ds_orgao_julgador_cargo'] = df_tjba['pje_ds_orgao_julgador_cargo'].str.upper()
df_tjba['pje_ds_natureza'] = df_tjba['pje_ds_natureza'].str.upper()
df_tjba['vm_ds_prioridade'] = df_tjba['vm_ds_prioridade'].str.upper()

# Retirando o timestamp
df_tjba['vm_dt_recebimento'] = df_tjba['vm_dt_recebimento'].dt.normalize()
df_tjba['vm_dt_movimento'] = df_tjba['vm_dt_movimento'].dt.normalize()

# Convertendo float para int
df_tjba['pje_vl_peso_processual'] = df_tjba['pje_vl_peso_processual'].astype('Int64')
df_tjba['pje_vl_peso_prioridade'] = df_tjba['pje_vl_peso_prioridade'].astype('Int64')

In [8]:
df_tjba.head()

,vm_processo,vm_dt_recebimento,vm_dt_movimento,vm_cod_comarca,vm_comarca,vm_cod_vara,vm_vara,vm_cod_cls_cnj,vm_classe,pje_ds_natureza,pje_cd_assunto_principal,pje_ds_assunto_principal,vm_cod_mov_cnj,vm_movimento,vm_competencia,vm_localizacao,vm_tp_proc,pje_id_orgao_julgador_cargo,pje_ds_orgao_julgador_cargo,vm_mapa_magistrado,vm_magistrado,vm_consulta,vm_situacao,vm_tempo_tramitacao,pje_vl_peso_processual,vm_prioridade,vm_ds_prioridade,pje_vl_peso_prioridade,vm_concluso,vm_sistema,vm_num_ativo
0,8003619-75.2022.8.05.0271,2022-09-19 00:00:00+00:00,2024-08-26 00:00:00+00:00,1255,VALENCA,11534,2ª VARA DE FEITOS DE REL DE CONS. CÍVEL E COMERCIAIS,65,ACAO CIVIL PUBLICA,CONHECIMENTO,10113,FLORA,11010,PROFERIDO DESPACHO DE MERO EXPEDIENTE,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,DIREÇÃO DE SECRETARIA,DIGITAL,1125.0,JUIZ DE DIREITO TITULAR,378.0,LEONARDO RULIAN CUSTODIO,DESPACHO,EM ANDAMENTO,0.0,4,N,None,0,0,PJEPG,1
1,8003068-15.2022.8.05.0039,2022-02-06 00:00:00+00:00,2024-09-19 00:00:00+00:00,115,CAMACARI,11453,2ª VARA DA FAZENDA PÚBLICA,1116,EXECUCAO FISCAL,EXECUÇÃO,6017,DÍVIDA ATIVA (EXECUÇÃO FISCAL),461,EXTINTO O PROCESSO POR AUSENCIA DAS CONDICOES DA ACAO,FAZENDA PUBLICA TRIBUTOS,DIREÇÃO DE SECRETARIA,DIGITAL,935.0,JUIZ DE DIREITO,166.0,DANIEL LIMA FALCAO,JULGAMENTO,BAIXADO,0.0,4,N,None,0,0,PJEPG,1
2,0088391-11.2003.8.05.0001,2003-07-18 00:00:00+00:00,2025-07-18 00:00:00+00:00,18,SALVADOR,111452,15ª VARA DA FAZENDA PÚBLICA,1116,EXECUCAO FISCAL,EXECUÇÃO,12989,EXECUÇÃO FISCAL,11010,PROFERIDO DESPACHO DE MERO EXPEDIENTE,FAZENDA PUBLICA ATOS ADMINISTRATIVOS,DIREÇÃO DE SECRETARIA,DIGITAL,2853.0,JUIZ DE DIREITO TITULAR,161.0,CRISTIANE MENEZES SANTOS BARRETO,DESPACHO,EM ANDAMENTO,0.0,4,N,None,0,0,PJEPG,1
3,0000087-73.2019.8.05.0066,2019-03-18 00:00:00+00:00,2025-07-09 00:00:00+00:00,1165,CONDEUBA,11754,VARA JURISDIÇÃO PLENA,283,ACAO PENAL PROCEDIMENTO ORDINARIO,CONHECIMENTO,11417,ESTUPRO DE VULNERÁVEL,11010,PROFERIDO DESPACHO DE MERO EXPEDIENTE,CRIME,DIREÇÃO DE SECRETARIA,DIGITAL,1740.0,JUIZ SUBSTITUTO,1492.0,CARLOS TIAGO SILVA ADAES,DESPACHO,EM ANDAMENTO,0.0,4,N,None,0,0,PJEPG,1
4,8055437-37.2021.8.05.0001,2021-05-28 00:00:00+00:00,2025-05-30 00:00:00+00:00,18,SALVADOR,11553,6ª VARA CÍVEL E COMERCIAL,7,PROCEDIMENTO COMUM CIVEL,CONHECIMENTO,9597,SEGURO,11010,PROFERIDO DESPACHO DE MERO EXPEDIENTE,CIVEL,DIREÇÃO DE SECRETARIA,DIGITAL,1165.0,JUIZ DE DIREITO TITULAR,750.0,CARLOS CARVALHO RAMOS DE CERQUEIRA JUNIOR,DESPACHO,EM ANDAMENTO,0.0,4,N,None,0,0,PJEPG,1


In [ ]:
#nomes_magistrados = df_tjba['vm_magistrado'].dropna().unique()
#
#df_magistrados['vm_nome'] = [
#    next((nome for nome in nomes_magistrados if str(nome).startswith(str(cadastro))), None) 
#    for cadastro in df_magistrados['Nome']
#]

In [ ]:
## Agora com os nomes ajustados podemos colocando a matrícula dos magistrados 
## que temos no cadastro no df principal
#df_tjba = df_tjba.merge(
#    df_magistrados[['Nome', 'Matrícula Funcional']], 
#    left_on='vm_magistrado', 
#    right_on='Nome', 
#    how='left'
#).rename(columns={'Matrícula Funcional': 'pt_matricula'})
#
## Removendo coluna Nome
#df_tjba.drop('Nome', axis=1, inplace=True)
#
## Pegando lista de colunas
#cols = list(df_tjba.columns)
#
## Removendo coluna
#cols.remove('pt_matricula')
#
## Achando posição, onde adicionar coluna
#pos = cols.index('vm_mapa_magistrado') + 1
#
## Adicionado coluna na posição escolhida
#cols.insert(pos, 'pt_matricula')
#
## Reordenado df
#df_tjba = df_tjba[cols]

### Análise Exploratória Conjunto de dados Final

In [9]:
# Quantidade de registro ao total por sistema
df_tjba['vm_sistema'].value_counts()

vm_sistema
PJEPG     11556834
SEEU        158333
SAJPG           71
SAIPRO           8
Name: count, dtype: int64

In [10]:
# Selecionando dados de 2024 e do PJE e registros somente com magistrados
df_tjba_fil = df_tjba[(df_tjba['vm_dt_movimento'].dt.year == 2024) & 
                    (df_tjba['vm_sistema'] == 'PJEPG') & 
                    (df_tjba['vm_magistrado'].isna() == False)]

In [13]:
# Verifica tipo de consultas
print('Tipos de consulta: ', df_tjba_fil['vm_consulta'].unique())
print('Quantidade de magistrados: ',df_tjba_fil['vm_magistrado'].nunique())
print('Quantidade de registros', df_tjba_fil.shape)
print('Quantidade de processos', df_tjba_fil['vm_processo'].nunique())

Tipos de consulta:  ['DESPACHO' 'JULGAMENTO' 'DECISAO']
Quantidade de magistrados:  624
Quantidade de registros (4225839, 31)
Quantidade de processos 2706715


In [20]:
# Magistrados por Comarca
df_comarca = (
    df_tjba_fil.groupby(['vm_cod_comarca', 'vm_comarca'])['vm_magistrado'].nunique().reset_index(name='qtd_magistrados')
)

df_comarca.head()


,vm_cod_comarca,vm_comarca,qtd_magistrados
0,18,SALVADOR,614
1,110,PORTO SEGURO,198
2,111,EUNAPOLIS,157
3,112,ITANHEM,20
4,114,BARRA,44


In [22]:
# Magistrados por Vara
df_vara = (
    df_tjba.groupby(['vm_cod_comarca', 'vm_comarca', 'vm_cod_vara', 'vm_vara'])['vm_magistrado']
    .nunique().reset_index(name='Qtd. Magistrado')
).rename(columns={
    'vm_cod_comarca': 'Cod. Comarca',
    'vm_comarca': 'Comarca',
    'vm_cod_vara': 'Cod. Vara',
    'vm_vara': 'Vara'
})

df_vara.head()

,Cod. Comarca,Comarca,Cod. Vara,Vara,Qtd. Magistrado
0,18,SALVADOR,1115,13ª VARA DA FAZENDA PÚBLICA,36
1,18,SALVADOR,1117,VARA DE ACIDENTES DE TRABALHO,60
2,18,SALVADOR,11352,[CEJUSC] PAI PRESENTE,2
3,18,SALVADOR,11458,"1 VARA DE SUCESSOES, ORFAOS INTERDITOS",81
4,18,SALVADOR,11459,"2 VARA DE SUCESSOES, ORFAOS INTERDITOS",72


### Tratando dados do Portal da Transparência

In [10]:
# Alguns nomes do df de cadastro (PT) não batem com o do TJBA, ajustando o cadastro por similaridade
df_magistrados['vm_nome'] = np.nan
nomes_mov = df_tjba['vm_magistrado'].dropna().unique()

for i, row in df_magistrados.iterrows():
    nome_cadastro = row['Nome']
    match, score, idx = process.extractOne(nome_cadastro, nomes_mov)
    
    if score > 85:  # limiar de confiança
        df_magistrados.at[i, 'vm_nome'] = match

In [11]:
df_magistrados.head()

,Nome,Matrícula Funcional,Cargo,Função de Confiança/ Cargo em Comissão,Lotação,Ato de Provimento,Data de Publicação,vm_nome
7,ABRAAO BARRETO CORDEIRO,9679782,JUIZ DE DIREITO,NaN,VARA CRIM JURI EXEC PENAIS MEN,PORT S/N,16/09/2013,ABRAAO BARRETO CORDEIRO
25,ADALBERTO LIMA BORGES FILHO,9698345,JUIZ DE DIREITO,NaN,JURISDICAO PLENA,PORT S/N,06/10/2021,ADALBERTO LIMA BORGES FILHO
69,ADERALDO DE MORAIS LEITE JUNIOR,9679871,JUIZ DE DIREITO,NaN,JURISDICAO PLENA,PORT S/N,16/09/2013,ADERALDO DE MORAIS LEITE JUNIOR
73,ADIANE JAQUELINE NEVES DA SILVA OLIVEIRA,9679812,JUIZ DE DIREITO,NaN,1ª VARA DOS SISTEMA DOS JUIZADOS ESPECIAIS,PORT S/N,16/09/2013,ADIANE JAQUELINE NEVES DA SILVA
74,ADIDA ALVES DOS SANTOS,8098654,JUIZ DE DIREITO,NaN,2ª VARA DA INFANCIA E JUVENTUDE,PORT S/N,01/12/2004,ADIDA ALVES DOS SANTOS


In [12]:
# Verificando se todos os nomes foram preenchidos
df_magistrados[df_magistrados['vm_nome'].isna()]

,Nome,Matrícula Funcional,Cargo,Função de Confiança/ Cargo em Comissão,Lotação,Ato de Provimento,Data de Publicação,vm_nome


In [ ]:
##
#for i, row, in df_magistrados.iterrows():
#    magistrado = row['vm_nome']
#    dados = df_tjba.loc[df_tjba['vm_magistrado'] == magistrado, ['vm_cod_comarca', 'vm_comarca', 'vm_cod_vara']]
#    
#    if not dados.empty:
#        df_magistrados.at[i, 'vm_cod_comarca'] = dados.iloc[0]['vm_cod_comarca']
#        df_magistrados.at[i, 'vm_comarca'] = dados.iloc[0]['vm_comarca']
#        df_magistrados.at[i, 'vm_cod_vara'] = dados.iloc[0]['vm_cod_vara']

In [ ]:
# Podemos ver que possivélmente temos magistrados em mais de uma vara.
df_tjba.loc[df_tjba['vm_magistrado'] == 'ADIANE JAQUELINE NEVES DA SILVA', ['vm_cod_comarca', 'vm_comarca', 'vm_cod_vara']]

,vm_cod_comarca,vm_comarca,vm_cod_vara
9377,1251,VITORIA DA CONQUISTA,11425
21317,1251,VITORIA DA CONQUISTA,11425
22420,1251,VITORIA DA CONQUISTA,11425
25255,1251,VITORIA DA CONQUISTA,11425
27194,1251,VITORIA DA CONQUISTA,11425
...,...,...,...
11586221,1251,VITORIA DA CONQUISTA,11425
11591347,1251,VITORIA DA CONQUISTA,11425
11598572,1251,VITORIA DA CONQUISTA,11425
11598918,1253,ITAPETINGA,11818
